In [ ]:
import pandas as pd
import numpy as np

In [29]:
from tqdm import tqdm


In [2]:
from sklearn.preprocessing import MultiLabelBinarizer

In [14]:

btdf = pd.read_parquet(r'C:/Users/ppaul/Documents/AI-strategies-papers-regulations-monitoring/data/s2orc/processed_big.parquet')
cit_df = pd.read_parquet(r'C:/Users/ppaul/Documents/AI-strategies-papers-regulations-monitoring/data/s2orc/big_ai_dataset.parquet')
cdf = cit_df[['paper_id','inbound_citations','outbound_citations']]
df = pd.merge(btdf,cdf,on='paper_id')

In [77]:
df = pd.read_parquet('meme_score.parquet')

sum inbound_citations that has meme of publication with this meme and\sum inbound_citations of publication with this meme
sum (pub that has a meme and does not have a meme-carrer in inbound_citations)/sum(pub that do not cite meme carrers)

In [15]:
meme_list = dict()
memes=[]
for chunks_list in df['noun_chunks_cleaned']:
    for chunk in chunks_list:
        if chunk not in meme_list:
            meme_list[chunk]=1
            memes.append(chunk)
        else:
            meme_list[chunk]+=1

In [78]:
df.head()

,paper_id,doc_lens,nouns,noun_chunks,lemmas,noun_chunks_cleaned,inbound_citations,outbound_citations,inbound_citations_clear,outbound_citations_clear
0,199668001,196,"[Road, construction, projects, territory, over...","[Road construction projects, the territory, th...","[road, construction, project, on, the, territo...","[road construction project, territory, republi...",[],"[106476531, 67083539, 36731616, 34616216, 1096...",[],[]
1,2879234,235,"[nets, CNNs, performance, history, approaches,...","[Convolutional neural nets, CNNs, remarkable p...","[convolutional, neural, net, (, cnn, ), have, ...","[convolutional neural net, cnn, remarkable per...","[11015941, 52160763, 199488257, 23874112, 4241...","[206592419, 215721, 10111903, 1003907, 3198903...","[340420, 471907, 250792]","[215721, 127386, 392527]"
2,17786914,120,"[results, benchmarks, modeling, speech, recogn...","[excellent results, benchmarks, acoustic model...","[recently, ,, deep, neural, networks(DNNs, ), ...","[excellent result, benchmark, acoustic modelin...",[198931159],"[9530137, 398770, 207168299, 14832074, 1799800...",[],"[398770, 299222]"
3,17432300,235,"[characters, structure, character, arrangement...","[East-Asian characters, a rich hierarchical st...","[east, -, asian, character, possess, a, rich, ...","[east asian character, rich hierarchical struc...","[15983137, 7625356, 15404413, 199472639, 47640...","[371064, 3246932, 8991475, 36725681, 32031694,...",[467570],[371064]
4,204957502,33,"[Internet, articles, multimedia, content, life...","[the Internet, countless articles, multimedia ...","[with, the, internet, become, widespread, ,, c...","[internet, countless article, multimedia conte...",[],[],[],[]


In [ ]:
#for i in df.iterrows:
#    memes[0] in i['noun_chunks_cleaned']#df[]all() not in df['outbound_citations']


: 

In [17]:
inc=[]
out=[]
for i,row in df.iterrows():
    cit =[]
    for cited in row['outbound_citations']:
        cited = int(cited)
        if cited in df['paper_id']:
            cit.append(cited)
        
    out.append(cit)
    cit=[]
    for citing in row['inbound_citations']:
        citing = int(citing)
        if citing in df['paper_id']:
            cit.append(citing)
    inc.append(cit)
df['inbound_citations_clear']=inc
df['outbound_citations_clear']=out

In [19]:
df.to_parquet('meme_score.parquet')

In [20]:
df['noun_chunks_cleaned'] = df['noun_chunks_cleaned'].apply(list)

In [107]:
df['noun_chunks_cleaned'].str.contains('a a b b',regex=False).sum()

1

In [79]:
enc = MultiLabelBinarizer(sparse_output=True)
memes_enc = enc.fit_transform(df['noun_chunks_cleaned'])

In [95]:
enc.classes_.tolist()

['',
 'a a a topology',
 'a a b b',
 'a a baseline approach',
 'a a baseline single classifier framework',
 'a a big pool',
 'a a brute force optimization',
 'a a cad system design',
 'a a classical rule base system',
 'a a classification tree',
 'a a clear functional separation',
 'a a comparison',
 'a a compound rule base expert system',
 'a a concert like condition',
 'a a contextual combination',
 'a a conventional feedforward multilayer neural network',
 'a a d tree expository',
 'a a data fidelity term',
 'a a datum drive initial segmentation',
 'a a decision theory model',
 'a a design',
 'a a design workflow',
 'a a domain level plan recognizer',
 'a a family',
 'a a fast prototype sdr platform',
 'a a fault detector',
 'a a first order approximation',
 'a a fuzzy expert system',
 'a a general image sentiment classifier',
 'a a generator g',
 'a a good feature extraction technique',
 'a a hybrid assemble architecture',
 'a a knowledge base',
 'a a learning framework',
 'a a los

Zbadałam częstości memów. 
Wykasowałam memy

OneHotEncoding memów w paperach cytowanych.

In [111]:
#a mask to apply to list of memes(nonexistent as of now)
memes_mask = np.nonzero(memes_enc.sum(axis=0)-1)[1]
memes_enc_cleaned = memes_enc[:,memes_mask]

<505000x846793 sparse matrix of type '<class 'numpy.int32'>'
	with 14533478 stored elements in Compressed Sparse Row format>

In [73]:
#frequency of memes
pd.DataFrame(np.sort(memes_enc.sum(axis=0))).T.value_counts()

1         3036974
2          398942
3          137087
4           70562
5           42965
           ...   
1836            1
1839            1
1840            1
1841            1
251321          1
Length: 2099, dtype: int64

In [124]:
df.index = df['paper_id']

In [220]:
df

,paper_id,doc_lens,nouns,noun_chunks,lemmas,noun_chunks_cleaned,inbound_citations,outbound_citations,inbound_citations_clear,outbound_citations_clear,memes_in_cited
paper_id,,,,,,,,,,,
199668001,199668001,196,"[Road, construction, projects, territory, over...","[Road construction projects, the territory, th...","[road, construction, project, on, the, territo...","[road construction project, territory, republi...",[],"[106476531, 67083539, 36731616, 34616216, 1096...",[],[],[]
2879234,2879234,235,"[nets, CNNs, performance, history, approaches,...","[Convolutional neural nets, CNNs, remarkable p...","[convolutional, neural, net, (, cnn, ), have, ...","[convolutional neural net, cnn, remarkable per...","[11015941, 52160763, 199488257, 23874112, 4241...","[206592419, 215721, 10111903, 1003907, 3198903...","[340420, 471907, 250792]","[215721, 127386, 392527]","[term, overall goal, numerous network activity..."
17786914,17786914,120,"[results, benchmarks, modeling, speech, recogn...","[excellent results, benchmarks, acoustic model...","[recently, ,, deep, neural, networks(DNNs, ), ...","[excellent result, benchmark, acoustic modelin...",[198931159],"[9530137, 398770, 207168299, 14832074, 1799800...",[],"[398770, 299222]","[very good computational efficiency, training ..."
17432300,17432300,235,"[characters, structure, character, arrangement...","[East-Asian characters, a rich hierarchical st...","[east, -, asian, character, possess, a, rich, ...","[east asian character, rich hierarchical struc...","[15983137, 7625356, 15404413, 199472639, 47640...","[371064, 3246932, 8991475, 36725681, 32031694,...",[467570],[371064],"[geometric consistency, correspondence, image,..."
204957502,204957502,33,"[Internet, articles, multimedia, content, life...","[the Internet, countless articles, multimedia ...","[with, the, internet, become, widespread, ,, c...","[internet, countless article, multimedia conte...",[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
127628365,127628365,179,"[models, strength, phenomena, reality, self, o...","[Abstract Probabilistic models, their strength...","[Abstract, Probabilistic, model, have, prove, ...","[abstract probabilistic model, strength, many ...",[],[],[],[],[]
11619477,11619477,189,"[AD, courses, patients, period, time, points, ...","[Alzheimer's Disease (AD, different courses, s...","[Alzheimer, 's, Disease, (, ad, ), can, take, ...","[alzheimer disease ad, different course, patie...",[204230004],"[34935848, 27355075, 9895278, 8461070, 3276119...",[],[],[]
5807289,5807289,217,"[Concurrency, bugs, software, testing, nature,...","[Concurrency bugs, software testing, their non...","[concurrency, bug, be, notoriously, difficult,...","[concurrency bug, software testing, non determ...","[15460152, 69949082, 5076939, 18091491, 678683...","[9005386, 17620776, 3439914, 978769, 2941443, ...",[],[],[]


In [222]:
memes_in_cited = []
#I want new column of unique noun chunks in all cited papers
for i,paper in tqdm(df.iterrows()):
    if paper['outbound_citations_clear'].size>0:
        #only memes in cited papers:
        c = df.iloc[paper['outbound_citations_clear']]['noun_chunks_cleaned']
        
        memes_in_cited.append(list(set(c.explode())))
    else:
        memes_in_cited.append(list(set()))

505000it [01:04, 7776.69it/s]


In [223]:
df['memes_in_cited']=memes_in_cited

In [243]:
a = df['memes_in_cited'].dropna()

In [245]:
a.str.len()

paper_id
2879234     96
17786914    99
17432300    83
62651912    32
62653139    27
            ..
11301469    24
53770387    48
6048192     27
15894398    28
15895702    15
Name: memes_in_cited, Length: 115089, dtype: int64

In [231]:
df['memes_in_cited'] = df['memes_in_cited'].dropna().apply(list)

In [250]:
all_memes = a.explode()

In [254]:
set(all_memes)

{'',
 'behavior tracking',
 'new graph learn algorithm',
 'standard svm',
 'its possible morphological constituent',
 'policy gradient technique',
 'expand operating space',
 'validity guide fuzzy cluster evaluation',
 'fertilizer',
 'coke oven gas collector coupling',
 'multi channel sample',
 'autonomy framework',
 'interactive play',
 'bpa',
 'model free base ipd controller',
 'phonetic control sequence',
 'specific artificial neural network paradigm',
 'generate index',
 'real time and accuracy index',
 'osana',
 'pattern evaluation',
 'fp cluster',
 'hardware intellectual property',
 'challenge toartificial intelligence',
 'combustible chemical residue',
 'significantly low perplexity',
 'other assessment method',
 'current lead automate va coding method',
 'combustor component',
 'particular source',
 'only authentication',
 'such demand application',
 'adequate believable behavior',
 'amateur skilled forgery',
 'main influence factor',
 'substantial information',
 'new research 

In [257]:
c_enc = MultiLabelBinarizer(sparse_output=True)
cited_memes_enc = c_enc.fit_transform(a)

TypeError: 'NoneType' object is not callable

parquet ilości noun chunków 

In [ ]:
cited_memes_enc_cleaned = cited_memes_enc[:,memes_mask]

In [ ]:
from tqdm import tqdm
#instead: two oneHotEncodings. First: which memes appear in paper. Second: which  memes appear in cited papers.
#p_spark 1: sum in papers axis of all ones of ohe1&ohe2.
#meme axis=0, papers axis=1

cited_memes_enc_cleaned.sum(axis=1)
(1-cited_memes_enc_cleaned).sum(axis=1)
cited_memes_enc_cleaned.multiply(memes_enc_cleaned).sum(axis=1)
(1-cited_memes_enc_cleaned).multiply(memes_enc_cleaned).sum(axis=1)

p_spark1 = []
p_spark2 = []
p_stick1 = []
p_stick2 = []
for key in tqdm(memes):
    #does it have meme? in OneHot: memes_enc
    c = df['noun_chunks_cleaned'].str.contains(key,regex=False)
    #does it cite meme carrers? in OneHot: all memes in citated
    c_spark = df['outbound_citations_clear'].str.contains(any(df[c]['paper_id']),regex=False)
    c_stick = ~c_spark
    

    p_spark1.append((c_spark&c).sum())
    p_spark2.append(c_spark.sum())
    p_stick1.append((c_stick&c).sum())
    p_stick2.append(c_stick.sum())

    
    

  0%|          | 5/3883767 [00:11<2498:41:11,  2.32s/it]


KeyboardInterrupt: 

: 